In [261]:
import pandas as pd
import numpy as np
import requests
from datetime import date
import datetime
from flask import Flask, render_template, request
from wtforms import Form, TextAreaField, validators
import pickle
import pymysql
import os
import numpy as np
import math
import time as tt

In [247]:
from __future__ import print_function
import mysql.connector
from mysql.connector import errorcode
import pymysql

In [248]:
import pandas as pd
from sqlalchemy import create_engine
from sqlalchemy.types import NVARCHAR, Float, Integer
from sqlalchemy import create_engine
from mysql.connector import (connection)

In [249]:
import folium
import folium.plugins
from branca.element import Figure

In [295]:
def do_calculation(car_num, creditNO, start_date, end_date):
    
#连接mysql服务器
  
    cnx = connection.MySQLConnection(user='', 
                                     password='',
                                     host='',
                                     database='')
    cursor = cnx.cursor()
    mysqlInfo = {
        "host": '',
        "user": '',
        "password": '',
        "database": '',
        "port": 3306,
        "charset": 'utf8'
    }
    engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % mysqlInfo, encoding='utf-8')
    conn = engine.connect()
    
    dtypedict = {
        'id': NVARCHAR(length=255),
        '车牌号': NVARCHAR(length=255),
        '取数开始时间': NVARCHAR(length=255),
        '取数截止时间': NVARCHAR(length=255),
        '打点开始时间': NVARCHAR(length=255),
        '打点截止时间': NVARCHAR(length=255),
        '取数条数': NVARCHAR(length=255),
        '调用时间': NVARCHAR(length=255),
        '项目名称': NVARCHAR(length=255),
        '经度': NVARCHAR(length=255),
        '纬度': NVARCHAR(length=255),
        '时间': NVARCHAR(length=255),
        '速度': NVARCHAR(length=255),
        '角度': NVARCHAR(length=255)
    }


# 中交api 取数
    url = ""
    payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
    headers = {
        'appKey': "",
        'Content-Type': "",
        'cache-control': "",
        'Postman-Token': ""
        }
    response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
    re = response.json()

#如果返回code ！= 200 的话证明时间超过7天,多次访问并整合结果
    if re['code'] != 200:
        hzb = pd.DataFrame()
        mxb = pd.DataFrame()
        start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

        s = start_date
        e = end_date

        end_date = start_date + datetime.timedelta(days=7)
#while 循环 check
        while end_date <= e:
    
            end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
            start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
            url = ""
            payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
            headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

            response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
            re = response.json()
            if len(re['data']['gpsInfo']) != 0:
                df = pd.DataFrame()
                df1= pd.DataFrame()

                df['车牌号'] = ""
                df['取数开始时间'] = ""
                df['取数截止时间'] = ""
                df['取数条数'] = ""
                df['调用时间'] = ""
                df['项目名称'] = ""
                df['打点开始时间'] = ""
                df['打点截止时间'] = ""
        
                df['车牌号'] = [re['data']['vehicleNumber']]
                df['取数开始时间'] = [str(start_date)]
                df['取数截止时间'] = [str(end_date)]
                df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
                df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
                df['取数条数'] = [str(len(re['data']['gpsInfo']))]
                df['调用时间'] = [str(datetime.datetime.now())]
                df['项目名称'] = [re['data']['creditNO']]

                cph = []
                longitude = []
                latitude = []
                time = []
                speed = []
                direction = []
        
                for i in range(len(re['data']['gpsInfo'])):
                    cp = re['data']['vehicleNumber']
                    cph.append(cp)
    
                    lg = re['data']['gpsInfo'][i]['long']
                    longitude.append(lg)
    
                    la = re['data']['gpsInfo'][i]['lat']
                    latitude.append(la)
    
                    tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                    time.append(tm)
    
                    spd = re['data']['gpsInfo'][i]['speed']
                    speed.append(spd)
    
                    di = re['data']['gpsInfo'][i]['direction']
                    direction.append(di)
              
                df1['车牌号'] = cph
                df1['经度'] = longitude
                df1['纬度'] = latitude
                df1['时间'] = time
                df1['速度'] = speed
                df1['角度'] = direction
    
                cnx = connection.MySQLConnection(user='', 
                                 password='',
                                 host='',
                                 database='')
        
                cursor = cnx.cursor()
                df.to_sql("汇总表", engine, if_exists='append', index=False, dtype=dtypedict)
                sql = """select * from 汇总表 where id=(select max(id) from 汇总表)"""
                cursor.execute(sql)
                data = cursor.fetchall()
                df2 = pd.DataFrame(list(data))
                
                idl = []
                for i in range(len(re['data']['gpsInfo'])):
                    ii = df2[0][0]
                    idl.append(ii)
                
                df1['汇总表id'] = idl
                df1.to_sql("明细表", engine, if_exists='append', index=False, dtype=dtypedict) 
                
                hzb = pd.concat([hzb,df])
                mxb = pd.concat([mxb,df1])
    
                start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
                end_date = start_date + datetime.timedelta(days=7)
            else:
                hzb = hzb
                mxb = mxb
    
                start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
                end_date = start_date + datetime.timedelta(days=7)
                
        else:
            end_date = e
            end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
            start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')

            url = ""
            payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
            headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

            response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
            re = response.json()
            if len(re['data']['gpsInfo']) != 0:
                df = pd.DataFrame()
                df1= pd.DataFrame()

                df['车牌号'] = ""
                df['取数开始时间'] = ""
                df['取数截止时间'] = ""
                df['取数条数'] = ""
                df['调用时间'] = ""
                df['项目名称'] = ""
                df['打点开始时间'] = ""
                df['打点截止时间'] = ""
        
                df['车牌号'] = [re['data']['vehicleNumber']]
                df['取数开始时间'] = [str(start_date)]
                df['取数截止时间'] = [str(end_date)]
                df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
                df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
                df['取数条数'] = [str(len(re['data']['gpsInfo']))]
                df['调用时间'] = [str(datetime.datetime.now())]
                df['项目名称'] = [re['data']['creditNO']]

                cph = []
                longitude = []
                latitude = []
                time = []
                speed = []
                direction = []
         
                for i in range(len(re['data']['gpsInfo'])):
                    cp = re['data']['vehicleNumber']
                    cph.append(cp)
    
                    lg = re['data']['gpsInfo'][i]['long']
                    longitude.append(lg)
    
                    la = re['data']['gpsInfo'][i]['lat']
                    latitude.append(la)
    
                    tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                    time.append(tm)
    
                    spd = re['data']['gpsInfo'][i]['speed']
                    speed.append(spd)
    
                    di = re['data']['gpsInfo'][i]['direction']
                    direction.append(di)
              
                df1['车牌号'] = cph
                df1['经度'] = longitude
                df1['纬度'] = latitude
                df1['时间'] = time
                df1['速度'] = speed
                df1['角度'] = direction
                cnx = connection.MySQLConnection(user='', 
                                 password='',
                                 host='',
                                 database='')
                cursor = cnx.cursor()
                df.to_sql("汇总表", engine, if_exists='append', index=False, dtype=dtypedict)
    
                sql = """select * from 汇总表 where id=(select max(id) from 汇总表)"""
                cursor.execute(sql)
                data = cursor.fetchall()
                df2 = pd.DataFrame(list(data))
    
                idl = []
                for i in range(len(re['data']['gpsInfo'])):
                    ii = df2[0][0]
                    idl.append(ii)
                
                df1['汇总表id'] = idl
                df1.to_sql("明细表", engine, if_exists='append', index=False, dtype=dtypedict) 
                
                hzb = pd.concat([hzb,df])
                mxb = pd.concat([mxb,df1])
                
                answer = """本次访问车辆结果如下:
                            最早打点时间:{start}
                            最晚打点时间:{end}
                            您的访问结果已经保存为excel格式至桌面,文件名称为 gps.xlsx.""".format(start = hzb.head(1)['打点开始时间'][0], end = hzb.tail(1)['打点截止时间'][0])
            else:
                hzb = hzb
                mxb = mxb

                answer = """本次访问车辆结果如下:
                            最早打点时间:{start}
                            最晚打点时间:{end}
                            您的访问结果已经保存为excel格式至桌面,文件名称为 gps.xlsx.""".format(start = hzb.head(1)['打点开始时间'][0], end = hzb.tail(1)['打点截止时间'][0])

#如果返回code = 200的话 证明 访问时间<= 7 天, 将返回数据输出为汇总表+明细表,但需要区分是否有数据   
    else:
        if len(re['data']['gpsInfo']) != 0: #先区分是否有数据
            df = pd.DataFrame()
            df1= pd.DataFrame()

            df['车牌号'] = ""
            df['取数开始时间'] = ""
            df['取数截止时间'] = ""
            df['取数条数'] = ""
            df['调用时间'] = ""
            df['项目名称'] = ""
            df['打点开始时间'] = ""
            df['打点截止时间'] = ""
        
            df['车牌号'] = [re['data']['vehicleNumber']]
            df['取数开始时间'] = [str(start_date)]
            df['取数截止时间'] = [str(end_date)]
            df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
            df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
            df['取数条数'] = [str(len(re['data']['gpsInfo']))]
            df['调用时间'] = [str(datetime.datetime.now())]
            df['项目名称'] = [re['data']['creditNO']]

            cph = []
            longitude = []
            latitude = []
            time = []
            speed = []
            direction = []
        
            for i in range(len(re['data']['gpsInfo'])):
                cp = re['data']['vehicleNumber']
                cph.append(cp)
    
                lg = re['data']['gpsInfo'][i]['long']
                longitude.append(lg)
    
                la = re['data']['gpsInfo'][i]['lat']
                latitude.append(la)
    
                tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                time.append(tm)
    
                spd = re['data']['gpsInfo'][i]['speed']
                speed.append(spd)
    
                di = re['data']['gpsInfo'][i]['direction']
                direction.append(di)
        
            df1['车牌号'] = cph
            df1['经度'] = longitude
            df1['纬度'] = latitude
            df1['时间'] = time
            df1['速度'] = speed
            df1['角度'] = direction
            cnx = connection.MySQLConnection(user='', 
                                 password='',
                                 host='',
                                 database='')
            cursor = cnx.cursor()
            df.to_sql("汇总表", engine, if_exists='append', index=False, dtype=dtypedict)
    
            sql = """select * from 汇总表 where id=(select max(id) from 汇总表)"""
            cursor.execute(sql)
            data = cursor.fetchall()
            df2 = pd.DataFrame(list(data))
    
            idl = []
            for i in range(len(re['data']['gpsInfo'])):
                ii = df2[0][0]
                idl.append(ii)
                
            df1['汇总表id'] = idl
            df1.to_sql("明细表", engine, if_exists='append', index=False, dtype=dtypedict)
                
            answer = "本次访问车辆"+"最早打点时间:"+df['打点开始时间'][0]+" "+"最晚打点时间:"+df['打点截止时间'][0]+" "+"您的访问结果已经保存为excel格式至桌面,文件名称为 gps.xlsx."
        else:
            answer = "您输入的日期范围内没有GPS数据"

#将两张表导出到桌面

    writer = pd.ExcelWriter('gps.xlsx', engine='xlsxwriter')
    df.to_excel(writer, sheet_name='汇总表')
    df1.to_excel(writer, sheet_name='明细表')
    writer.save()

# 在output 中插入线路地图

    n = len(mxb)
    cor = []
    df1 = mxb.reset_index()
    for i in range(n):
        cor.append([df1['纬度'][i],df1['经度'][i]])
    fig=Figure(height=550,width=750)
    m=folium.Map(location=cor[0],zoom_start=8)
    fig.add_child(m)

    f=folium.FeatureGroup(df['车牌号'][0])
    line_1=folium.vector_layers.PolyLine(cor,popup='<b>Path of {vehicle}</b>'.format(vehicle=df['车牌号'][0]),tooltip=df['车牌号'][0],color='blue',weight=10).add_to(f)

    f.add_to(m)
    folium.LayerControl().add_to(m)
    
    m.save("templates/map.html")

    return answer

In [ ]:
from flask import Flask, request
from flask import render_template

app = Flask(__name__)
app.config["DEBUG"] = False


@app.route("/", methods=["GET", "POST"])
def adder_page():
    errors = ""
    if request.method == "POST":
        car_num = str(request.form["car_num"])
        creditNO = str(request.form["creditNO"])
        start_date = str(request.form["start_date"])
        end_date = str(request.form["end_date"])
        result = do_calculation(car_num, creditNO, start_date, end_date)
    
#post之后的返回结果       
        return '''
            <html>
                <body>
                    <p><a href="/">点击这里再次进行查询</a>
                    <p>{result1}</p>
                    
                    <p>访问结果为: {result2}</p>
                    
                </body>
            </html>
        '''.format(result1 = result ,result2 = render_template('map.html'))

#网页主界面的返回结果
    return '''
        <html>
            <body>
                {errors}
                <p>请按照以下要求填写参数</p>
                <form method="post" action=".">
                    <p>车牌号 例如: 陕AN1907 </p>
                    <p><input name="car_num" /></p>
                    <p>项目名称 例如: 大件</p>
                    <p><input name="creditNO" /></p>
                    <p>起始日期 例如: 2020-08-29 20:00:00</p>
                    <p><input name="start_date" /></p>
                    <p>截止日期 例如: 2020-08-29 23:00:00</p>
                    <p><input name="end_date" /></p>
                    <p>备注: 起始日期与截止日期为在6个自然月内日期，否则无法获取数据</p>
                    <p><input type="submit" value="提交并获取数据" /></p>
                </form>
            </body>
        </html>
    '''.format(errors=errors)

#往主机运行app
if __name__ == "__main__":
    app.run(debug=False)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [12/Jan/2021 18:00:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Jan/2021 18:01:03] "POST / HTTP/1.1" 200 -


In [ ]:
#连接数据库代码 勿删 备用 
from sqlalchemy import create_engine
mysqlInfo = {
    "host": '',
    "user": '',
    "password": '',
    "database": '',
    "port": 3306,
    "charset": 'utf8'
}
engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % mysqlInfo, encoding='utf-8')


In [ ]:
#建表代码 勿删 备用 
    SQL1 = """CREATE TABLE `汇总表1` (
  
  `id` int(11) NOT NULL AUTO_INCREMENT,
  `车牌号` varchar(20) DEFAULT NULL,
  `取数开始时间` varchar(20) DEFAULT NULL,
  `取数截止时间` varchar(20) DEFAULT NULL,
  `取数条数` varchar(20) DEFAULT NULL,
  `调用时间` varchar(30) DEFAULT NULL,
  `项目名称` varchar(20) DEFAULT NULL,
  PRIMARY KEY (`id`)

    ) ENGINE=InnoDB DEFAULT CHARSET=utf8;
    """
    cursor.execute(SQL1)
    
    SQL2 = """CREATE TABLE `明细表` (
  
  `id` int(11) NOT NULL AUTO_INCREMENT, 
  `车牌号` varchar(20) DEFAULT NULL,
  `经度` varchar(20) DEFAULT NULL,
  `纬度` varchar(20) DEFAULT NULL,
  `时间` varchar(20) DEFAULT NULL,
  `速度` varchar(20) DEFAULT NULL,
  `角度` varchar(20) DEFAULT NULL,
  `汇总表id` int(11) DEFAULT NULL,
  PRIMARY KEY (`id`)
  
    ) ENGINE=InnoDB DEFAULT CHARSET=utf8;
    """
    cursor.execute(SQL2)

In [21]:
#中交api 接口访问代码 勿删 备用 
car_num = ''
creditNO = ''
start_date = '2020-12-02 20:00:00'
end_date = '2020-12-15 20:01:00'

url = ""
payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
re = response.json()



In [104]:
# 去掉中交api7天的限制 while loop that is something truly useful ~ 
car_num = ''
creditNO = ''
start_date = '2020-11-02 20:00:00'
end_date = '2020-11-22 20:00:00'

hzb = pd.DataFrame()
mxb = pd.DataFrame()

start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

s = start_date
e = end_date

end_date = start_date + datetime.timedelta(days=7)

while end_date < e:
    
    end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
    start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
    url = ""
    payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
    headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

    response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
    re = response.json()
    
    df = pd.DataFrame()
    df1= pd.DataFrame()

    df['车牌号'] = ""
    df['取数开始时间'] = ""
    df['取数截止时间'] = ""
    df['取数条数'] = ""
    df['调用时间'] = ""
    df['项目名称'] = ""
    df['打点开始时间'] = ""
    df['打点截止时间'] = ""
        
    df['车牌号'] = [re['data']['vehicleNumber']]
    df['取数开始时间'] = [str(start_date)]
    df['取数截止时间'] = [str(end_date)]
    df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
    df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
    df['取数条数'] = [str(len(re['data']['gpsInfo']))]
    df['调用时间'] = [str(datetime.datetime.now())]
    df['项目名称'] = [re['data']['creditNO']]

    cph = []
    longitude = []
    latitude = []
    time = []
    speed = []
    direction = []
        
    
    for i in range(len(re['data']['gpsInfo'])):
        cp = re['data']['vehicleNumber']
        cph.append(cp)
    
        lg = re['data']['gpsInfo'][i]['long']
        longitude.append(lg)
    
        la = re['data']['gpsInfo'][i]['lat']
        latitude.append(la)
    
        tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
        time.append(tm)
    
        spd = re['data']['gpsInfo'][i]['speed']
        speed.append(spd)
    
        di = re['data']['gpsInfo'][i]['direction']
        direction.append(di)
              
    df1['车牌号'] = cph
    df1['经度'] = longitude
    df1['纬度'] = latitude
    df1['时间'] = time
    df1['速度'] = speed
    df1['角度'] = direction
    
    hzb = pd.concat([hzb,df])
    mxb = pd.concat([mxb,df1])
    
    start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
    end_date = start_date + datetime.timedelta(days=7)
else:
    end_date = e
    end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
    start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
    url = ""
    payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
    headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

    response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
    re = response.json()
    
    df = pd.DataFrame()
    df1= pd.DataFrame()

    df['车牌号'] = ""
    df['取数开始时间'] = ""
    df['取数截止时间'] = ""
    df['取数条数'] = ""
    df['调用时间'] = ""
    df['项目名称'] = ""
    df['打点开始时间'] = ""
    df['打点截止时间'] = ""
        
    df['车牌号'] = [re['data']['vehicleNumber']]
    df['取数开始时间'] = [str(start_date)]
    df['取数截止时间'] = [str(end_date)]
    df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
    df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
    df['取数条数'] = [str(len(re['data']['gpsInfo']))]
    df['调用时间'] = [str(datetime.datetime.now())]
    df['项目名称'] = [re['data']['creditNO']]

    cph = []
    longitude = []
    latitude = []
    time = []
    speed = []
    direction = []
         
    for i in range(len(re['data']['gpsInfo'])):
        cp = re['data']['vehicleNumber']
        cph.append(cp)
    
        lg = re['data']['gpsInfo'][i]['long']
        longitude.append(lg)
    
        la = re['data']['gpsInfo'][i]['lat']
        latitude.append(la)
    
        tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
        time.append(tm)
    
        spd = re['data']['gpsInfo'][i]['speed']
        speed.append(spd)
    
        di = re['data']['gpsInfo'][i]['direction']
        direction.append(di)
              
    df1['车牌号'] = cph
    df1['经度'] = longitude
    df1['纬度'] = latitude
    df1['时间'] = time
    df1['速度'] = speed
    df1['角度'] = direction
    
    hzb = pd.concat([hzb,df])
    mxb = pd.concat([mxb,df1])

In [ ]:
#导出两个表 代码 
car_num = ''
creditNO = ''
start_date = '2020-12-29 20:00:00'
end_date = '2020-12-30 23:00:00'

url = ""
payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
re = response.json()

df = pd.DataFrame()
df1= pd.DataFrame()

df['id'] = ""
df['车牌号'] = ""
df['取数开始时间'] = ""
df['取数截止时间'] = ""
df['取数条数'] = ""
df['调用时间'] = ""
df['项目名称'] = ""

df['id'] = [1]
df['车牌号'] = [re['data']['vehicleNumber']]
df['取数开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
df['取数截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
df['取数条数'] = [str(len(re['data']['gpsInfo']))]
df['调用时间'] = [str(datetime.now())]
df['项目名称'] = [re['data']['creditNO']]

cph = []
longitude = []
latitude = []
time = []
speed = []
direction = []
idl = []

for i in range(len(re['data']['gpsInfo'])):
    cp = re['data']['vehicleNumber']
    cph.append(cp)
    
    lg = re['data']['gpsInfo'][i]['long']
    longitude.append(lg)
    
    la = re['data']['gpsInfo'][i]['lat']
    latitude.append(la)
    
    tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
    time.append(tm)
    
    spd = re['data']['gpsInfo'][i]['speed']
    speed.append(spd)
    
    di = re['data']['gpsInfo'][i]['direction']
    direction.append(di)
    
    
    
df1['车牌号'] = cph
df1['经度'] = longitude
df1['纬度'] = latitude
df1['时间'] = time
df1['速度'] = speed
df1['角度'] = direction

writer = pd.ExcelWriter('gps.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='汇总表')
df1.to_excel(writer, sheet_name='明细表')



In [19]:
#地图
import folium
import folium.plugins
from branca.element import Figure
cor = []
for i in range(len(df1)):
    cor.append([df1['纬度'][i],df1['经度'][i]])
fig=Figure(height=550,width=750)
m=folium.Map(location=cor[0],zoom_start=10)
fig.add_child(m)

f=folium.FeatureGroup(df['车牌号'][0])
line_1=folium.vector_layers.PolyLine(cor,popup='<b>Path of {vehicle}</b>'.format(vehicle=df['车牌号'][0]),tooltip=df['车牌号'][0],color='blue',weight=10).add_to(f)

f.add_to(m)
folium.LayerControl().add_to(m)
m

In [141]:
#while loop codes redesign the api
car_num = ''
creditNO = ''
start_date = '2020-11-05 20:00:00'
end_date = '2020-11-26 23:00:00'
url = ""
payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
re = response.json()

if re['code'] != 200:
    hzb = pd.DataFrame()
    mxb = pd.DataFrame()
    
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

    s = start_date
    e = end_date

    end_date = start_date + datetime.timedelta(days=7)

    while end_date < e:
        end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
        start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
        url = ""
        payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
        headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

        response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
        re = response.json()
        if len(re['data']['gpsInfo']) != 0:
            df = pd.DataFrame()
            df1= pd.DataFrame()

            df['车牌号'] = ""
            df['取数开始时间'] = ""
            df['取数截止时间'] = ""
            df['取数条数'] = ""
            df['调用时间'] = ""
            df['项目名称'] = ""
            df['打点开始时间'] = ""
            df['打点截止时间'] = ""
        
            df['车牌号'] = [re['data']['vehicleNumber']]
            df['取数开始时间'] = [str(start_date)]
            df['取数截止时间'] = [str(end_date)]
            df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
            df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
            df['取数条数'] = [str(len(re['data']['gpsInfo']))]
            df['调用时间'] = [str(datetime.datetime.now())]
            df['项目名称'] = [re['data']['creditNO']]

            cph = []
            longitude = []
            latitude = []
            time = []
            speed = []
            direction = []
        
    
            for i in range(len(re['data']['gpsInfo'])):
                cp = re['data']['vehicleNumber']
                cph.append(cp)
    
                lg = re['data']['gpsInfo'][i]['long']
                longitude.append(lg)
    
                la = re['data']['gpsInfo'][i]['lat']
                latitude.append(la)
    
                tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                time.append(tm)
    
                spd = re['data']['gpsInfo'][i]['speed']
                speed.append(spd)
    
                di = re['data']['gpsInfo'][i]['direction']
                direction.append(di)
              
            df1['车牌号'] = cph
            df1['经度'] = longitude
            df1['纬度'] = latitude
            df1['时间'] = time
            df1['速度'] = speed
            df1['角度'] = direction
    
            hzb = pd.concat([hzb,df])
            mxb = pd.concat([mxb,df1])
    
            start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
            end_date = start_date + datetime.timedelta(days=7)
        else:
            hzb = hzb
            mxb = mxb
            
            start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
            end_date = start_date + datetime.timedelta(days=7)
    else:
        end_date = e
        end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
        start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
        url = ""
        payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
        headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

        response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
        re = response.json()
        if len(re['data']['gpsInfo']) != 0:
            df = pd.DataFrame()
            df1= pd.DataFrame()

            df['车牌号'] = ""
            df['取数开始时间'] = ""
            df['取数截止时间'] = ""
            df['取数条数'] = ""
            df['调用时间'] = ""
            df['项目名称'] = ""
            df['打点开始时间'] = ""
            df['打点截止时间'] = ""
        
            df['车牌号'] = [re['data']['vehicleNumber']]
            df['取数开始时间'] = [str(start_date)]
            df['取数截止时间'] = [str(end_date)]
            df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
            df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
            df['取数条数'] = [str(len(re['data']['gpsInfo']))]
            df['调用时间'] = [str(datetime.datetime.now())]
            df['项目名称'] = [re['data']['creditNO']]

            cph = []
            longitude = []
            latitude = []
            time = []
            speed = []
            direction = []
         
            for i in range(len(re['data']['gpsInfo'])):
                cp = re['data']['vehicleNumber']
                cph.append(cp)
    
                lg = re['data']['gpsInfo'][i]['long']
                longitude.append(lg)
    
                la = re['data']['gpsInfo'][i]['lat']
                latitude.append(la)
    
                tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                time.append(tm)
    
                spd = re['data']['gpsInfo'][i]['speed']
                speed.append(spd)
    
                di = re['data']['gpsInfo'][i]['direction']
                direction.append(di)
              
            df1['车牌号'] = cph
            df1['经度'] = longitude
            df1['纬度'] = latitude
            df1['时间'] = time
            df1['速度'] = speed
            df1['角度'] = direction
            
            hzb = pd.concat([hzb,df])
            mxb = pd.concat([mxb,df1])
        else:
            hzb = hzb
            mxb = mxb
    df = hzb
    df1 = mxb
#如果返回code = 200的话就将返回数据输出为汇总表+明细表,但需要区分是否有数据   
else:
    if len(re['data']['gpsInfo']) != 0:
        df = pd.DataFrame()
        df1= pd.DataFrame()

        df['车牌号'] = ""
        df['取数开始时间'] = ""
        df['取数截止时间'] = ""
        df['取数条数'] = ""
        df['调用时间'] = ""
        df['项目名称'] = ""
        df['打点开始时间'] = ""
        df['打点截止时间'] = ""
        
        df['车牌号'] = [re['data']['vehicleNumber']]
        df['取数开始时间'] = [str(start_date)]
        df['取数截止时间'] = [str(end_date)]
        df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
        df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
        df['取数条数'] = [str(len(re['data']['gpsInfo']))]
        df['调用时间'] = [str(datetime.datetime.now())]
        df['项目名称'] = [re['data']['creditNO']]

        cph = []
        longitude = []
        latitude = []
        time = []
        speed = []
        direction = []
        
    
        for i in range(len(re['data']['gpsInfo'])):
            cp = re['data']['vehicleNumber']
            cph.append(cp)
    
            lg = re['data']['gpsInfo'][i]['long']
            longitude.append(lg)
    
            la = re['data']['gpsInfo'][i]['lat']
            latitude.append(la)
    
            tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
            time.append(tm)
    
            spd = re['data']['gpsInfo'][i]['speed']
            speed.append(spd)
    
            di = re['data']['gpsInfo'][i]['direction']
            direction.append(di)
        
        
        
        df1['车牌号'] = cph
        df1['经度'] = longitude
        df1['纬度'] = latitude
        df1['时间'] = time
        df1['速度'] = speed
        df1['角度'] = direction
            
        answer = "本次访问车辆最早时间与最晚时间如下"+"\n"+"最早打点时间:"+df['打点开始时间'][0]+" "+"最晚打点时间:"+df['打点截止时间'][0]+"\n"+"您的访问结果已经保存为excel格式至桌面,文件名称为 gps.xlsx."
    else:
        answer = "您输入的日期范围内没有GPS数据"
writer = pd.ExcelWriter('gps.xlsx', engine='xlsxwriter')
df.to_excel(writer, sheet_name='汇总表')
df1.to_excel(writer, sheet_name='明细表')

In [220]:
# checking block - entire app
car_num = ''
creditNO = ''
start_date = '2020-11-05 20:00:00'
end_date = '2020-11-26 23:00:00'

cnx = connection.MySQLConnection(user='', 
                                 password='',
                                 host='',
                                 database='')
cursor = cnx.cursor()
mysqlInfo = {
    "host": '',
    "user": '',
    "password": '',
    "database": '',
    "port": 3306,
    "charset": 'utf8'
}
engine = create_engine('mysql+pymysql://%(user)s:%(password)s@%(host)s:%(port)d/%(database)s?charset=utf8' % mysqlInfo, encoding='utf-8')
conn = engine.connect()
    
dtypedict = {
    'id': NVARCHAR(length=255),
    '车牌号': NVARCHAR(length=255),
    '取数开始时间': NVARCHAR(length=255),
    '取数截止时间': NVARCHAR(length=255),
    '打点开始时间': NVARCHAR(length=255),
    '打点截止时间': NVARCHAR(length=255),
    '取数条数': NVARCHAR(length=255),
    '调用时间': NVARCHAR(length=255),
    '项目名称': NVARCHAR(length=255),
    '经度': NVARCHAR(length=255),
    '纬度': NVARCHAR(length=255),
    '时间': NVARCHAR(length=255),
    '速度': NVARCHAR(length=255),
    '角度': NVARCHAR(length=255)
}


# 中交api 取数
url = ""
payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }
response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
re = response.json()

#如果返回code ！= 200 的话证明时间超过7天,多次访问并整合结果
if re['code'] != 200:
    hzb = pd.DataFrame()
    mxb = pd.DataFrame()
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S')
    end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')

    s = start_date
    e = end_date

    end_date = start_date + datetime.timedelta(days=7)

    while end_date < e:
    
        end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
        start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
    
        url = ""
        payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
        headers = {
            'appKey': "",
            'Content-Type': "",
            'cache-control': "",
            'Postman-Token': ""
            }

        response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
        re = response.json()
        if len(re['data']['gpsInfo']) != 0:
            df = pd.DataFrame()
            df1= pd.DataFrame()

            df['车牌号'] = ""
            df['取数开始时间'] = ""
            df['取数截止时间'] = ""
            df['取数条数'] = ""
            df['调用时间'] = ""
            df['项目名称'] = ""
            df['打点开始时间'] = ""
            df['打点截止时间'] = ""
        
            df['车牌号'] = [re['data']['vehicleNumber']]
            df['取数开始时间'] = [str(start_date)]
            df['取数截止时间'] = [str(end_date)]
            df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
            df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
            df['取数条数'] = [str(len(re['data']['gpsInfo']))]
            df['调用时间'] = [str(datetime.datetime.now())]
            df['项目名称'] = [re['data']['creditNO']]

            cph = []
            longitude = []
            latitude = []
            time = []
            speed = []
            direction = []
        
            for i in range(len(re['data']['gpsInfo'])):
                cp = re['data']['vehicleNumber']
                cph.append(cp)
    
                lg = re['data']['gpsInfo'][i]['long']
                longitude.append(lg)
    
                la = re['data']['gpsInfo'][i]['lat']
                latitude.append(la)
    
                tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                time.append(tm)
    
                spd = re['data']['gpsInfo'][i]['speed']
                speed.append(spd)
    
                di = re['data']['gpsInfo'][i]['direction']
                direction.append(di)
              
            df1['车牌号'] = cph
            df1['经度'] = longitude
            df1['纬度'] = latitude
            df1['时间'] = time
            df1['速度'] = speed
            df1['角度'] = direction
    
            df.to_sql("汇总表", engine, if_exists='append', index=False, dtype=dtypedict)
            sql = """select * from 汇总表 order by id desc limit 1"""
            cursor.execute(sql)
            data = cursor.fetchall()
            df2 = pd.DataFrame(list(data))
                
            idl = []
            for i in range(len(re['data']['gpsInfo'])):
                ii = df2[0][0]
                idl.append(ii)
                
            df1['汇总表id'] = idl
            df1.to_sql("明细表", engine, if_exists='append', index=False, dtype=dtypedict) 
                
            hzb = pd.concat([hzb,df])
            mxb = pd.concat([mxb,df1])
    
            start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
            end_date = start_date + datetime.timedelta(days=7)
        else:
            hzb = hzb
            mxb = mxb
    
            start_date = datetime.datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S')
            end_date = start_date + datetime.timedelta(days=7)
                
    else:
        end_date = e
        end_date = end_date.strftime('%Y-%m-%d %H:%M:%S')
        start_date = start_date.strftime('%Y-%m-%d %H:%M:%S')
        url = "https://panda.glp168.com/vehicle/track"
        payload = '\n{"vehicleNumber":"' + car_num +'","creditNO":"'+ creditNO + '","queryStartDay":"'+ start_date + '","queryEndDay":"'+ end_date + '"},\n'
        headers = {
        'appKey': "externalApi001",
        'Content-Type': "application/json",
        'cache-control': "no-cache",
        'Postman-Token': "99e3e52b-a31c-4a40-8e8a-3e45107e1fd9"
        }

        response = requests.request("POST", url, data=payload.encode('utf-8'), headers=headers)
        re = response.json()
        if len(re['data']['gpsInfo']) != 0:
            df = pd.DataFrame()
            df1= pd.DataFrame()

            df['车牌号'] = ""
            df['取数开始时间'] = ""
            df['取数截止时间'] = ""
            df['取数条数'] = ""
            df['调用时间'] = ""
            df['项目名称'] = ""
            df['打点开始时间'] = ""
            df['打点截止时间'] = ""
        
            df['车牌号'] = [re['data']['vehicleNumber']]
            df['取数开始时间'] = [str(start_date)]
            df['取数截止时间'] = [str(end_date)]
            df['打点开始时间'] = [str(re['data']['gpsInfo'][0]['time']['year'])+"-"+str(re['data']['gpsInfo'][0]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][0]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][0]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][0]['time']['minute'])+':'+str(re['data']['gpsInfo'][0]['time']['second'])]
            df['打点截止时间'] = [str(re['data']['gpsInfo'][-1]['time']['year'])+"-"+str(re['data']['gpsInfo'][-1]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][-1]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][-1]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][-1]['time']['minute'])+':'+str(re['data']['gpsInfo'][-1]['time']['second'])]
            df['取数条数'] = [str(len(re['data']['gpsInfo']))]
            df['调用时间'] = [str(datetime.datetime.now())]
            df['项目名称'] = [re['data']['creditNO']]

            cph = []
            longitude = []
            latitude = []
            time = []
            speed = []
            direction = []
         
            for i in range(len(re['data']['gpsInfo'])):
                cp = re['data']['vehicleNumber']
                cph.append(cp)
    
                lg = re['data']['gpsInfo'][i]['long']
                longitude.append(lg)
    
                la = re['data']['gpsInfo'][i]['lat']
                latitude.append(la)
    
                tm = str(re['data']['gpsInfo'][i]['time']['year'])+"-"+str(re['data']['gpsInfo'][i]['time']['monthValue'])+"-"+str(re['data']['gpsInfo'][i]['time']['dayOfMonth'])+" "+str(re['data']['gpsInfo'][i]['time']['hour'])+ ":" + str(re['data']['gpsInfo'][i]['time']['minute'])+':'+str(re['data']['gpsInfo'][i]['time']['second'])
                time.append(tm)
    
                spd = re['data']['gpsInfo'][i]['speed']
                speed.append(spd)
    
                di = re['data']['gpsInfo'][i]['direction']
                direction.append(di)
              
            df1['车牌号'] = cph
            df1['经度'] = longitude
            df1['纬度'] = latitude
            df1['时间'] = time
            df1['速度'] = speed
            df1['角度'] = direction
                
            df.to_sql("汇总表", engine, if_exists='append', index=False, dtype=dtypedict)
            sql = """select * from 汇总表 order by id desc limit 1"""
            cursor.execute(sql)
            data = cursor.fetchall()
            df2 = pd.DataFrame(list(data))
    
            idl = []
            for i in range(len(re['data']['gpsInfo'])):
                ii = df2[0][0]
                idl.append(ii)
                
            df1['汇总表id'] = idl
            df1.to_sql("明细表", engine, if_exists='append', index=False, dtype=dtypedict) 
                
            hzb = pd.concat([hzb,df])
            mxb = pd.concat([mxb,df1])
        else:
            hzb = hzb
            mxb = mxb
    
    answer = "本次访问车辆最早时间与最晚时间如下"+"\n"+"最早打点时间:"+hzb.head(1)['打点开始时间'][0]+" "+"最晚打点时间:"+df.tail(1)['打点截止时间'][0]+"\n"+"您的访问结果已经保存为excel格式至桌面,文件名称为 gps.xlsx."